In [38]:
import requests
import json
from PyPDF2 import PdfReader
import os

In [39]:
def get_papers(ids):
    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        headers={'x-api-key':'umacVekm3o29skv66Q3JxrXT6jAoad7m2P1RwZg0'},
        params={'fields': 'title,isOpenAccess,openAccessPdf,authors'},
        json={"ids": ids}
    )
    return r.json()

In [42]:
r = requests.get(
    'https://api.semanticscholar.org/graph/v1/paper/search/bulk?year=2023&venue=AAAI',
    headers={'x-api-key':'umacVekm3o29skv66Q3JxrXT6jAoad7m2P1RwZg0'}
)

# Iterate in 500 paper chunks and get the paper details
response_data = r.json()
papers = []
for i in range(0, len(response_data["data"]), 500):
    papers += get_papers([p['paperId'] for p in response_data["data"][i:i+500]])

paper_download = [(p['title'], p['openAccessPdf']) for p in papers if p['isOpenAccess']]

[{'paperId': '0000f2eb98405f7d4044761c1707a199ac6189bc', 'title': 'When Are Two Lists Better than One?: Benefits and Harms in Joint Decision-making', 'isOpenAccess': True, 'openAccessPdf': {'url': 'https://arxiv.org/pdf/2308.11721', 'status': 'CLOSED'}, 'authors': [{'authorId': '122495956', 'name': 'Kate Donahue'}, {'authorId': '24194181', 'name': 'Kostas Kollias'}, {'authorId': '144979147', 'name': 'Sreenivas Gollapudi'}]}, {'paperId': '000d34a7e19984bf4a0eb32dfcac414a9d16143d', 'title': 'Object Attribute Matters in Visual Question Answering', 'isOpenAccess': False, 'openAccessPdf': None, 'authors': [{'authorId': '2236932833', 'name': 'Peize Li'}, {'authorId': '84109730', 'name': 'Q. Si'}, {'authorId': '143655088', 'name': 'Peng Fu'}, {'authorId': '2237093798', 'name': 'Zheng Lin'}, {'authorId': '2237134412', 'name': 'Yan Wang'}]}, {'paperId': '001bddae7fd798a7422ecc1b0818752e2401fbc2', 'title': 'Learning Multi-Object Positional Relationships via Emergent Communication', 'isOpenAccess

In [43]:
print(papers)
print(paper_download[0])

[{'paperId': '0000f2eb98405f7d4044761c1707a199ac6189bc', 'title': 'When Are Two Lists Better than One?: Benefits and Harms in Joint Decision-making', 'isOpenAccess': True, 'openAccessPdf': {'url': 'https://arxiv.org/pdf/2308.11721', 'status': 'CLOSED'}, 'authors': [{'authorId': '122495956', 'name': 'Kate Donahue'}, {'authorId': '24194181', 'name': 'Kostas Kollias'}, {'authorId': '144979147', 'name': 'Sreenivas Gollapudi'}]}, {'paperId': '000d34a7e19984bf4a0eb32dfcac414a9d16143d', 'title': 'Object Attribute Matters in Visual Question Answering', 'isOpenAccess': False, 'openAccessPdf': None, 'authors': [{'authorId': '2236932833', 'name': 'Peize Li'}, {'authorId': '84109730', 'name': 'Q. Si'}, {'authorId': '143655088', 'name': 'Peng Fu'}, {'authorId': '2237093798', 'name': 'Zheng Lin'}, {'authorId': '2237134412', 'name': 'Yan Wang'}]}, {'paperId': '001bddae7fd798a7422ecc1b0818752e2401fbc2', 'title': 'Learning Multi-Object Positional Relationships via Emergent Communication', 'isOpenAccess

In [44]:
def download_pdf(url, save_path):
    """Download a PDF from a URL and save it to a specified path."""
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

def extract_first_page_text(pdf_path):
    """Extract text from the first page of a PDF."""
    reader = PdfReader(pdf_path)
    first_page = reader.pages[0]
    return first_page.extract_text()

def extract_paper_data(download_urls):
    texts = {}
    for url in download_urls:
        # Extract file name from URL and create a local path
        file_name = url[1]["url"].split('/')[-1]
        file_path = os.path.join(os.getcwd(), file_name)

        # Download the PDF
        download_pdf(url[1]["url"], file_path)

        # Extract and print text from the first page
        text = extract_first_page_text(file_path)
        texts = {url[0]: text}
        # Remove the PDF file
        os.remove(file_path)
    return texts

In [45]:
texts = extract_paper_data(paper_download)

In [46]:
from mlx_lm import load, generate
model, tokenizer = load("mlx-community/Meta-Llama-3.1-8B-8bit")

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [47]:
keys = list(texts.keys())
results = {}
for i in range(0, 20):
    test_text = texts[keys[i]]
    prompt_start = """Extract the authors and their affiliations from the first page of the paper and put them into a list containing of json arrays like {"author":"firstname,lastname", "affiliation":"university"} Text: """
    prompt_end = " Sure, here is the List of Json arrays extracted from the text:"
    prompt = prompt_start + test_text + prompt_end
    generated = generate(model, tokenizer, prompt, max_tokens=200, repetition_penalty=1.0)
    results[keys[i]] = generated

IndexError: list index out of range

In [ ]:
output = generated.split("]")[0] + "]"
output = output.replace("\n", "")
output = "[" + output.split("[")[1]
print(output)
output = json.loads(output)
print(output)

[{"author":"Kate Donahue", "affiliation":"Cornell University"}, {"author":"Sreenivas Gollapudi", "affiliation":"Google"}, {"author":"Kostas Kollias", "affiliation":"Google"}]
[{'author': 'Kate Donahue', 'affiliation': 'Cornell University'}, {'author': 'Sreenivas Gollapudi', 'affiliation': 'Google'}, {'author': 'Kostas Kollias', 'affiliation': 'Google'}]
